In [15]:
import pandas as pd
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

# 1. Загрузка данных
# Пример данных: запросы и соответствующие ответы с метками релевантности
data = [
    {
        "query": "Добрый день! Подскажите пожалуйста, мне нужно создать заявку в директуме о переводе корпоративного номера тел на физ лицо в связи с увольнением,, я обновила директум, но не могу там найти как сделать эту заявку! Помогите пожалуйста оперативно. 12 декабря последний рабочий день мой. Спасибо",
        "answers": [
            {
                "text": "Для получения SIM-карты и других манипуляций, касающихся мобильной связи, заполните заявку по ссылке:",
                "label": 1,
            },
            {"text": "Оформите обращение в команду поддержки Directum:", "label": 0},
            {
                "text": "Для передачи оборудования оформите заявку по ссылке:",
                "label": 0,
            },
        ],
    },
    {
        "query": "Коллеги, добрый день! Клиент как обычно скинул заказы в PDF, но они почему то теперь не читаются: Обработать заказы нужно срочно, поэтому заявка через Элму не актуальна. Цена вопроса около 20 млн. Помогите, пжл, разобраться, что случилось. Проблема не только на моей стороне, так же читаются файлы и у моей коллеги. С ее слов, на прошлой неделе у нее была такая же проблема, но она успела обработать заказы. Позже она зашла, чтоб проверить и файлы стали нечитабельны.",
        "answers": [
            {
                "text": "Для получения поддержки по программному обеспечению заполните заявку по ссылке:",
                "label": 1,
            },
            {
                "text": "Для обслуживания принтера заполните заявку по ссылке:",
                "label": 0,
            },
            {
                "text": "Оформите обращение в команду поддержки Directum:",
                "label": 0,
            },
            {
                "text": "Поднимите данный запрос, если необходимо разработать новый ЗНО, доработать уже существующее решение на платформе ELMA365",
                "label": 0,
            },
        ],
    },
    {
        "query": "Установка What's App Коллеги, добрый день!\r\nПодскажите, пожалуйста, вы можете помочь установить приложение What’s App на планшет? Извините, если задавал этот вопрос уже. Приложение необходимо для оперативной связи с партнёрами.\r\n\r\n \r\n\r\nС наилучшими пожеланиями,\r\n\r\nКрупин Илья\r\n\r\nМладший специалист по поддержке и аналитике систем/ Advocacy\r\n\r\n* / Ilia.Krupin@ru.nestle.com <mailto:Ilia.Krupin@ru.nestle.com> \r\n\r\nПожалуйста, примите во внимание, что с 01.01.2025 адрес моей корпоративной электронной почты – Ilia.Krupin@nestle.ru <mailto:Ilia.Krupin@nestle.ru> \r\n\r\n \r\n\r\n\r\n\r\n \r\n",
        "answers": [
            {
                "text": "Использование некорпоративных мессенджеров запрещено.",
                "label": 1,
            }
        ],
    },
    {
        "query": "PDP форма Коллеги, добрый день!\r\n\r\nНе могу редактировать форму PDP. Не дает написать комментарий к целям, когда написала, нажимаю «ок», и ничего не сохраняется.\r\nПодскажите, пожалуйста, что делать.\r\n\r\nСпасибо\r\n\r\n \r\n\r\nKind regards,\r\n\r\n\r\nYasmina Khankan\r\n\r\nJunior Financial analyst\r\n\r\nSupply Chain Controlling\r\n\r\nMoscow, Paveletskaya Sq.2/1\r\n\r\nM.: +7 (985) 912-08-60\r\n\r\n \r\n\r\n\r\n\r\n \r\n",
        "answers": [{"text": "Приложите скриншот ошибки, пожалуйста.", "label": 1}],
    },
    {
        "query": "не могу подключиться PDP Добрый день! Нахожусь на удаленке и, не могу подключиться к PDP, прошу помочь мне с этим вопросом.",
        "answers": [{"text": "Приложите скриншот ошибки, пожалуйста.", "label": 1}],
    },
    {
        "query": "Логин пароль Коллеги, добрый день! \r\nПожалуйста, пришлите логин и пароль для нового сотрудника Селькиной Олеси\r\nЕе номер телефона  +7 926 176-30-71\r\n\r\nСкачайте Outlook для iOS <https://eur02.safelinks.protection.outlook.com/?url=https%3A%2F%2Faka.ms%2Fo0ukef&data=05%7C02%7CService.Desk%40ru.nestle.com%7C19c30184761743ebc34d08dd23fe1966%7C12a3af23a7694654847f958f3d479f4a%7C0%7C0%7C638706296425144245%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C0%7C%7C%7C&sdata=h3Re2JVbWgKOmjIwfk8hyfxvdTOF9iPfkSdxXKHFrAg%3D&reserved=0> ",
        "answers": [
            {
                "text": "Для получения SIM-карты и других манипуляций, касающихся мобильной связи, заполните заявку по ссылке:",
                "label": 0,
            }
        ],
    },
    
]

# 2. Подготовка данных для обучения
train_examples = []
for item in data:
    query = item["query"]
    for answer in item["answers"]:
        train_examples.append(
            InputExample(texts=[query, answer["text"]], label=answer["label"])
        )

In [16]:
# 3. Загрузка предобученной модели Sentence-BERT
model = SentenceTransformer("./sentence_bert_service_desk_model")

# 4. Настройка и дообучение модели
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=1)
train_loss = losses.CosineSimilarityLoss(model)

# Дообучение модели
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=3,
    warmup_steps=100,
    show_progress_bar=True,
)

  0%|          | 0/21 [00:00<?, ?it/s]

{'train_runtime': 97.1092, 'train_samples_per_second': 0.216, 'train_steps_per_second': 0.216, 'train_loss': 0.20483184996105375, 'epoch': 3.0}


In [19]:
from sklearn.metrics.pairwise import cosine_similarity

# 5. Оценка модели
# Пример оценки модели на новых данных
test_query = [
    "Коллеги, добрый день! Клиент как обычно скинул заказы в PDF, но они почему то теперь не читаются: Обработать заказы нужно срочно, поэтому заявка через Элму не актуальна. Цена вопроса около 20 млн. Помогите, пжл, разобраться, что случилось. Проблема не только на моей стороне, так же читаются файлы и у моей коллеги. С ее слов, на прошлой неделе у нее была такая же проблема, но она успела обработать заказы. Позже она зашла, чтоб проверить и файлы стали нечитабельны.",
    "Добрый день! Подскажите пожалуйста, мне нужно создать заявку в директуме о переводе корпоративного номера тел на физ лицо в связи с увольнением,, я обновила директум, но не могу там найти как сделать эту заявку! Помогите пожалуйста оперативно. 12 декабря последний рабочий день мой. Спасибо",
]
test_answers = [
    "Для получения SIM-карты и других манипуляций, касающихся мобильной связи, заполните заявку по ссылке:",
    "Оформите обращение в команду поддержки Directum:",
    "Для передачи оборудования оформите заявку по ссылке:",
    "Для получения поддержки по программному обеспечению заполните заявку по ссылке:",
]

answer_embeddings = model.encode(test_answers)

for i in range(2):
    query_embedding = model.encode(test_query[i])
    # Вычисление косинусного сходства
    cosine_scores = cosine_similarity([query_embedding], answer_embeddings)[0]

    # Сортировка ответов по сходству
    sorted_answers = sorted(
        zip(test_answers, cosine_scores), key=lambda x: x[1], reverse=True
    )
    print(f"Question: {test_query[i]}")
    # Вывод отсортированных ответов
    for answer, score in sorted_answers:
        print(f"Answer: {answer}, score: {score}")
    print()

Question: Коллеги, добрый день! Клиент как обычно скинул заказы в PDF, но они почему то теперь не читаются: Обработать заказы нужно срочно, поэтому заявка через Элму не актуальна. Цена вопроса около 20 млн. Помогите, пжл, разобраться, что случилось. Проблема не только на моей стороне, так же читаются файлы и у моей коллеги. С ее слов, на прошлой неделе у нее была такая же проблема, но она успела обработать заказы. Позже она зашла, чтоб проверить и файлы стали нечитабельны.
Answer: Для получения поддержки по программному обеспечению заполните заявку по ссылке:, Score: 0.13863997161388397
Answer: Оформите обращение в команду поддержки Directum:, Score: -0.0037203244864940643
Answer: Для получения SIM-карты и других манипуляций, касающихся мобильной связи, заполните заявку по ссылке:, Score: -0.0699332058429718
Answer: Для передачи оборудования оформите заявку по ссылке:, Score: -0.0953928530216217

Question: Добрый день! Подскажите пожалуйста, мне нужно создать заявку в директуме о перев

In [20]:
model.save("./sentence_bert_service_desk_model_finetuned")

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]